In [1]:
from sympy import init_printing; init_printing();
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from silkpy.surface import ParametricSurface
from sympy import symbols, sin, cos, pi, cot, Array
from silkpy.sympy_utility import dot
u, v = symbols('u, v', real=True)

In [3]:
surf_choice = 'torus'
if surf_choice=='cylindrical':
    R = symbols('R', positive=True)
    s = ParametricSurface([u, v], [R*cos(u), R*sin(u), v])
elif surf_choice=='cone':
    w = symbols('omega', real=True)
    s = ParametricSurface([u, v], [v*cos(u), v*sin(u), v*cot(w)])
elif surf_choice=='Mobius':
    theta = symbols('theta', real=True)
    s = ParametricSurface([theta, v],
        Array([cos(theta),                sin(theta),                0                ]) + 
        Array([sin(theta/2) * cos(theta), sin(theta/2) * sin(theta), cos(theta/2)]) * v)
elif surf_choice=='torus':
    a, r = symbols('a, r', real=True)
    s = ParametricSurface([u, v], [ (a+r*cos(u)) * cos(v), (a+r*cos(u)) * sin(v), r*sin(u)])

In [4]:
s.metric_tensor.tensor().simplify()
s.metric_tensor.change_config('uu').tensor()
s.christoffel_symbol.tensor().simplify()
r_u, r_v = s.expr().diff(u), s.expr().diff(v); r_u, r_v
a, b = r_u, r_v
s.weingarten_matrix
Wa = s.weingarten_transform(a) 
Wb = s.weingarten_transform(b)
dot(Wa, b), dot(a, Wb)
s.K_H()
s.prin_curvature_and_vector

/home/wenyin/pyvenv/vsym2/lib/python3.8/site-packages/IPython/lib/latextools.py:126: MatplotlibDeprecationWarning: 
The to_png function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
/home/wenyin/pyvenv/vsym2/lib/python3.8/site-packages/IPython/lib/latextools.py:126: MatplotlibDeprecationWarning: 
The to_rgba function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
/home/wenyin/pyvenv/vsym2/lib/python3.8/site-packages/IPython/lib/latextools.py:126: MatplotlibDeprecationWarning: 
The to_mask function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
/home/wenyin/pyvenv/vsym2/lib/python3.8/site-packages/IPython/lib/latextools.py:126: M

⎡ 2                 ⎤
⎢r          0       ⎥
⎢                   ⎥
⎢                  2⎥
⎣0   (a + r⋅cos(u)) ⎦

⎡1                  ⎤
⎢──         0       ⎥
⎢ 2                 ⎥
⎢r                  ⎥
⎢                   ⎥
⎢           1       ⎥
⎢0   ───────────────⎥
⎢                  2⎥
⎣    (a + r⋅cos(u)) ⎦

⎡                            ⎡               -r⋅sin(u)  ⎤⎤
⎢⎡0            0          ⎤  ⎢     0        ────────────⎥⎥
⎢⎢                        ⎥  ⎢              a + r⋅cos(u)⎥⎥
⎢⎢   (a + r⋅cos(u))⋅sin(u)⎥  ⎢                          ⎥⎥
⎢⎢0  ─────────────────────⎥  ⎢ -r⋅sin(u)                ⎥⎥
⎢⎣             r          ⎦  ⎢────────────       0      ⎥⎥
⎣                            ⎣a + r⋅cos(u)              ⎦⎦

([-r⋅sin(u)⋅cos(v)  -r⋅sin(u)⋅sin(v)  r⋅cos(u)], [-(a + r⋅cos(u))⋅sin(v)  (a +
 r⋅cos(u))⋅cos(v)  0])

TypeError: 'ImmutableDenseNDimArray' object is not callable

In [ ]:
# from silkpy.surface.draw import draw_surface_plotly

# if surf_choice=='cylindrical':
#     R = 1.0
#     s = ParametricSurface([u, v], [R*cos(u), R*sin(u), v])
# elif surf_choice=='cone':
#     w = float(pi) / 4
#     s = ParametricSurface([u, v], [v*cos(u), v*sin(u), v*cot(w)
#     draw_surface_plotly(s, domain=[(-2*float(pi), 2*float(pi)), (4, 6)])

